# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235398044357                   -0.50    7.0         
  2   -7.250302103116       -1.83       -1.40    1.0   11.1ms
  3   -7.251149204793       -3.07       -2.18    1.0   12.0ms
  4   -7.251231078782       -4.09       -2.11    3.0   17.7ms
  5   -7.251334927132       -3.98       -2.68    1.0   13.4ms
  6   -7.251337986810       -5.51       -3.16    1.0   13.3ms
  7   -7.251338681379       -6.16       -3.56    1.0   13.6ms
  8   -7.251338788768       -6.97       -3.98    2.0   18.6ms
  9   -7.251338797274       -8.07       -4.51    1.0   13.7ms
 10   -7.251338798462       -8.93       -4.89    2.0   15.9ms
 11   -7.251338798689       -9.64       -5.39    2.0   16.0ms
 12   -7.251338798702      -10.89       -5.94    1.0   13.7ms
 13   -7.251338798704      -11.81       -6.07    2.0   15.9ms
 14   -7.251338798704      -12.11       -6.84    1.0   15.2ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06653809902973797
[ Info: Arnoldi iteration step 2: normres = 0.3885259546437622
[ Info: Arnoldi iteration step 3: normres = 0.6899581120594744
[ Info: Arnoldi iteration step 4: normres = 0.49421942858680673
[ Info: Arnoldi iteration step 5: normres = 0.5018189539214835
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.13e-01, 6.00e-02, 3.83e-01, 2.98e-01, 1.10e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3008589952519847
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.78e-02, 7.65e-02, 2.51e-01, 8.70e-02, 1.11e-01)
[ Info: Arnoldi iteration step 7: normres = 0.07627329728603004
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.03e-03, 6.72e-03, 1.67e-02, 3.38e-02, 5.71e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13157501494014096
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.93e-